# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# What test is appropriate for this problem? Does CLT apply?

T test is appropriate for this problem, standard deviation is not known. 

Although the distribtion is binomial, but sample proportions are normally distributed and hence CTL is applicable.

# What are the null and alternate hypotheses?

Null hypotheses - There is no differnce in callback between white and black sounding resumes.
Alternate hypothese - There is a difference in callback between white and black sounding resumes.

# Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

### T test and Margin of Error

In [6]:
w_len = len(w)
b_len = len(b)


pro_w = sum(w.call) / w_len
pro_b = sum(b.call) / b_len

var_w = pro_w*(1-pro_w)
var_b = pro_b*(1-pro_b)

std_dev = np.sqrt((var_w/w_len)+(var_b/b_len))
mean_diff = pro_w - pro_b
t_score = mean_diff/std_dev

print("T Score "+str(t_score))

T Score 4.11555043573


In [7]:
# calculate t critical
t_critical = stats.t.ppf(0.975,df=w_len-1)

# Calculate margin of error
moe = t_critical * std_dev

# 95% confidence interval
conf_interval = [mean_diff-moe , mean_diff+moe]

pvalue = (1-stats.t.cdf(t_score, df=b_len-1))*2


print("Margin of Error "+str(moe))
print('95% confidence interval: '+str(conf_interval))
print('P-Value '+str(pvalue))

Margin of Error 0.0152627157139896
95% confidence interval: [0.016770138495455986, 0.047295569923435185]
P-Value 3.9910745856852614e-05


In [8]:
stats.ttest_ind(w.call, b.call,equal_var=False)

Ttest_indResult(statistic=4.114705290861751, pvalue=3.9429415136459197e-05)

P-value from the T test is almost zero and hence we can reject null hypothesis. We can conclude that there is significant difference between white and black sounding resumes.

### Bootstrap method

In [33]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2


def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def diff_of_prop(data_1, data_2):
    
    p1 = np.sum(data_1 == 1)/len(data_1)
    p2 = np.sum(data_2 == 1)/len(data_2)
    
    return abs(p1-p2)

In [34]:
# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w.call, b.call, diff_of_prop, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= (pro_w - pro_b)) / len(perm_replicates)

# Print the result
print('p-value ='+str(p))

p-value =0


P-value from bootstrap method is zero.  As it is alreay concluded in T test, null hypothesis can be rejected and conclude that there is significant difference between white and black sounding resumes.

# Write a story describing the statistical significance in the context or the original problem.
# Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Given only white sounding and black sounding names, we were able to conclude that there is a difference callback. But name can be associated with different factors. Hence other variables need to be considered to determine whether there is a racial bias.